In [24]:
"""
Transaction-Level Data Summary Generator
=========================================
Reads a CSV file with ~87,000 transaction rows and produces a
richly formatted multi-sheet Excel summary workbook.
"""

import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from pathlib import Path

# Get current working directory (EDA folder)
BASE_DIR = Path.cwd()

# Go one level up
PROJECT_ROOT = BASE_DIR.parent

INPUT_PATH = PROJECT_ROOT / "outputs" / "transactions_additional_features.parquet"
OUTPUT_PATH = PROJECT_ROOT /"EDA"/ "transactions_summary.xlsx"

print(PROJECT_ROOT)

c:\Users\VISHNUPRIYA\OneDrive\Desktop\Freelancing\AIGEN\smartsentry_aml_model


In [17]:
print(f"📂 Loading data from: {INPUT_PATH}")

df = pd.read_parquet(INPUT_PATH)

print(f"✅ Loaded {len(df):,} rows × {df.shape[1]} columns")

TOTAL = len(df)

📂 Loading data from: c:\Users\VISHNUPRIYA\OneDrive\Desktop\Freelancing\AIGEN\smartsentry_aml_model\outputs\transactions_additional_features.parquet
✅ Loaded 429,040 rows × 188 columns


In [25]:
df.head().to_csv("check_columns_txns.csv")

In [5]:
df.dtypes

transaction_id                 object
customer_id                    object
sender_account_id              object
receiver_account_id            object
beneficiary_id                 object
                               ...   
customer_risk_rating_num      float64
kyc_level_num                 float64
fraud_intensity_score_raw     float64
fraud_intensity_score         float64
fis_band                     category
Length: 188, dtype: object

In [18]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
#df["date"] = pd.to_datetime(df["date"], errors="coerce")

In [19]:

# ── Helper functions ────────────────────────────────────────────────────────────

def pct_df(series, label):
    d = series.value_counts().reset_index()
    d.columns = [label, "Count"]
    d["Percentage (%)"] = (d["Count"] / TOTAL * 100).round(2)
    return d

def stats_df(series, label):
    d = series.describe().round(4).reset_index()
    d.columns = ["Statistic", label]
    return d

def flag_summary(series, label, mapping=None):
    d = series.value_counts().reset_index()
    d.columns = [label, "Count"]
    if mapping:
        d[label] = d[label].map(mapping)
    d["Percentage (%)"] = (d["Count"] / TOTAL * 100).round(2)
    return d

def rule_summary(rule_cols, df):
    rows = []
    for col in rule_cols:
        fired = int(df[col].sum())
        rows.append({
            "Rule Name": col.replace("rule_", "").replace("_", " ").title(),
            "Column": col,
            "Times Fired": fired,
            "Percentage (%)": round(fired / TOTAL * 100, 2),
        })
    return pd.DataFrame(rows).sort_values("Times Fired", ascending=False).reset_index(drop=True)

# ── Styling constants ──────────────────────────────────────────────────────────

HDR_FILL  = PatternFill("solid", start_color="1F3864")
SUB_FILL  = PatternFill("solid", start_color="2E75B6")
HDR2_FILL = PatternFill("solid", start_color="BDD7EE")
ALT_FILL  = PatternFill("solid", start_color="D9E1F2")
WHITE     = PatternFill("solid", start_color="FFFFFF")
RED_FILL  = PatternFill("solid", start_color="FFE0E0")
THIN = Border(
    left=Side(style="thin", color="B8CCE4"),
    right=Side(style="thin", color="B8CCE4"),
    top=Side(style="thin", color="B8CCE4"),
    bottom=Side(style="thin", color="B8CCE4"),
)

def hdr(ws, row, col, val, size=11, fill=HDR_FILL, color="FFFFFF"):
    c = ws.cell(row=row, column=col, value=val)
    c.font = Font(name="Arial", bold=True, size=size, color=color)
    c.fill = fill
    c.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    c.border = THIN
    return c

def dat(ws, row, col, val, alt=False, bold=False, center=False, red=False):
    c = ws.cell(row=row, column=col, value=val)
    c.font = Font(name="Arial", size=10, bold=bold, color="1F2D3D")
    c.fill = RED_FILL if red else (ALT_FILL if alt else WHITE)
    c.alignment = Alignment(horizontal="center" if center else "left", vertical="center")
    c.border = THIN
    return c

def write_block(ws, sr, sc, title, dataframe, flag_high=None):
    """
    Write a titled table block at (sr, sc).
    flag_high: column name where value > threshold triggers red fill.
    Returns the next available row.
    """
    ec = sc + len(dataframe.columns) - 1
    ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
    hdr(ws, sr, sc, title, size=10, fill=SUB_FILL)
    ws.row_dimensions[sr].height = 22

    for ci, col_name in enumerate(dataframe.columns, start=sc):
        hdr(ws, sr + 1, ci, col_name, size=9, fill=HDR2_FILL, color="1F2D3D")

    for ri, (_, row_data) in enumerate(dataframe.iterrows()):
        alt = ri % 2 == 1
        for ci, val in enumerate(row_data, start=sc):
            is_red = False
            if flag_high and dataframe.columns[ci - sc] == "Percentage (%)" and isinstance(val, float) and val > flag_high:
                is_red = True
            dat(ws, sr + 2 + ri, ci, val, alt=alt, center=(ci > sc), red=is_red)

    # Auto column widths
    for ci in range(sc, ec + 1):
        cl = get_column_letter(ci)
        max_w = max(len(str(ws.cell(r, ci).value or "")) for r in range(sr, sr + 2 + len(dataframe)))
        ws.column_dimensions[cl].width = min(max(max_w + 4, 14), 42)

    return sr + 2 + len(dataframe) + 2

def write_fraud_example(ws, sr, sc, ftype, example_df, signals, descriptions):
    """
    Writes one fraud type example block containing:
      - Type name + description header
      - Reasoning bullets (why this tx is flagged)
      - Transaction field/value table
    Returns next available row.
    """
    display_name, description = descriptions.get(
        ftype, (ftype.replace("_"," ").title(), "No description available.")
    )
    ORANGE_FILL = PatternFill("solid", start_color="F4B942")
    GREEN_FILL  = PatternFill("solid", start_color="E2EFDA")
    PURPLE_FILL = PatternFill("solid", start_color="E8D5F5")

    # ── Section title (fraud type name) ────────────────────────────────
    ec = sc + 1
    ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
    c = ws.cell(row=sr, column=sc, value=f"🔴  {display_name}")
    c.font = Font(name="Arial", bold=True, size=12, color="FFFFFF")
    c.fill = PatternFill("solid", start_color="C00000")
    c.alignment = Alignment(horizontal="left", vertical="center")
    c.border = THIN
    ws.row_dimensions[sr].height = 24
    sr += 1

    # ── Description ────────────────────────────────────────────────────
    ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
    c = ws.cell(row=sr, column=sc, value=description)
    c.font = Font(name="Arial", italic=True, size=9, color="1F2D3D")
    c.fill = PatternFill("solid", start_color="FFF2CC")
    c.alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)
    c.border = THIN
    ws.row_dimensions[sr].height = 42
    sr += 1

    # ── "Why flagged?" sub-header ──────────────────────────────────────
    ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
    c = ws.cell(row=sr, column=sc, value="⚠️  Why is this transaction flagged?")
    c.font = Font(name="Arial", bold=True, size=10, color="FFFFFF")
    c.fill = ORANGE_FILL
    c.alignment = Alignment(horizontal="left", vertical="center")
    c.border = THIN
    ws.row_dimensions[sr].height = 20
    sr += 1

    # ── Signal bullets ─────────────────────────────────────────────────
    for i, signal in enumerate(signals):
        ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
        c = ws.cell(row=sr, column=sc, value=f"  {'✦'} {signal}")
        c.font = Font(name="Arial", size=9, color="1F2D3D")
        c.fill = ALT_FILL if i % 2 == 1 else WHITE
        c.alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)
        c.border = THIN
        ws.row_dimensions[sr].height = 16
        sr += 1

    # ── "Example transaction" sub-header ──────────────────────────────
    ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
    c = ws.cell(row=sr, column=sc, value="📋  Representative Transaction (highest risk score in this typology)")
    c.font = Font(name="Arial", bold=True, size=10, color="FFFFFF")
    c.fill = PURPLE_FILL
    c.font = Font(name="Arial", bold=True, size=10, color="1F2D3D")
    c.alignment = Alignment(horizontal="left", vertical="center")
    c.border = THIN
    ws.row_dimensions[sr].height = 20
    sr += 1

    # ── Field / Value table ────────────────────────────────────────────
    hdr(ws, sr, sc,     "Field", size=9, fill=HDR2_FILL, color="1F2D3D")
    hdr(ws, sr, sc + 1, "Value", size=9, fill=HDR2_FILL, color="1F2D3D")
    sr += 1
    for ri, row_data in example_df.iterrows():
        alt = ri % 2 == 1
        dat(ws, sr, sc,     row_data["Field"], alt=alt)
        dat(ws, sr, sc + 1, row_data["Value"], alt=alt)
        ws.row_dimensions[sr].height = 15
        sr += 1

    # ── Column widths ──────────────────────────────────────────────────
    ws.column_dimensions[get_column_letter(sc)].width     = 32
    ws.column_dimensions[get_column_letter(sc + 1)].width = 48

    return sr + 2  # gap between examples


# ══════════════════════════════════════════════════════════════════════════════
# BUILD SUMMARIES
# ══════════════════════════════════════════════════════════════════════════════

# ── 1. Transaction Overview ──────────────────────────────────────────────────
fraud_count    = int(df["label"].sum()) if "label" in df.columns else 0
legit_count    = TOTAL - fraud_count
fraud_pct      = round(fraud_count / TOTAL * 100, 2)
total_amount   = df["amount"].sum()
avg_amount     = df["amount"].mean()
median_amount  = df["amount"].median()
max_amount     = df["amount"].max()

overview_kpis = pd.DataFrame({
    "KPI": [
        "Total Transactions", "Fraud Transactions", "Legitimate Transactions",
        "Fraud Rate (%)", "Total Amount (INR)", "Avg Amount (INR)",
        "Median Amount (INR)", "Max Amount (INR)",
        "Unique Customers", "Unique Accounts (Sender)",
        "Unique Devices", "Unique Beneficiaries",
        "Date Range Start", "Date Range End",
    ],
    "Value": [
        f"{TOTAL:,}", f"{fraud_count:,}", f"{legit_count:,}",
        f"{fraud_pct}%", f"₹{total_amount:,.2f}", f"₹{avg_amount:,.2f}",
        f"₹{median_amount:,.2f}", f"₹{max_amount:,.2f}",
        df["customer_id"].nunique() if "customer_id" in df.columns else "N/A",
        df["sender_account_id"].nunique() if "sender_account_id" in df.columns else "N/A",
        df["device_id"].nunique() if "device_id" in df.columns else "N/A",
        df["beneficiary_id"].nunique() if "beneficiary_id" in df.columns else "N/A",
        str(df["timestamp"].min())[:10] if "timestamp" in df.columns else "N/A",
        str(df["timestamp"].max())[:10] if "timestamp" in df.columns else "N/A",
    ]
})

# ── 2. Fraud Type breakdown ──────────────────────────────────────────────────
fraud_type_df = pct_df(df["fraud_type"], "Fraud Type") if "fraud_type" in df.columns else pd.DataFrame()

# ── 3. Transaction characteristics ───────────────────────────────────────────
channel_df     = pct_df(df["channel"], "Channel") if "channel" in df.columns else pd.DataFrame()
txn_type_df    = pct_df(df["transaction_type"], "Transaction Type") if "transaction_type" in df.columns else pd.DataFrame()
debit_df       = pct_df(df["debit_credit"], "Debit/Credit") if "debit_credit" in df.columns else pd.DataFrame()
cash_df        = flag_summary(df["cash_flag"], "Cash Flag", {0: "Non-Cash", 1: "Cash"}) if "cash_flag" in df.columns else pd.DataFrame()
amount_stats   = stats_df(df["amount"], "Amount (INR)") if "amount" in df.columns else pd.DataFrame()

# ── 4. Temporal distribution ──────────────────────────────────────────────────
hour_df        = pct_df(df["hour"], "Hour of Day") if "hour" in df.columns else pd.DataFrame()
dow_map        = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}
dow_df         = pct_df(df["day_of_week"].map(dow_map) if "day_of_week" in df.columns else df["day_of_week"], "Day of Week") if "day_of_week" in df.columns else pd.DataFrame()
month_df       = pct_df(df["month"], "Month") if "month" in df.columns else pd.DataFrame()
night_df       = flag_summary(df["is_night"], "Is Night", {0:"Day",1:"Night"}) if "is_night" in df.columns else pd.DataFrame()
weekend_df     = flag_summary(df["is_weekend"], "Is Weekend", {0:"Weekday",1:"Weekend"}) if "is_weekend" in df.columns else pd.DataFrame()
biz_df         = flag_summary(df["is_business_hours"], "Business Hours", {0:"Non-Business",1:"Business"}) if "is_business_hours" in df.columns else pd.DataFrame()

# ── 5. Customer & Account Profile ─────────────────────────────────────────────
occ_df         = pct_df(df["occupation"], "Occupation") if "occupation" in df.columns else pd.DataFrame()
kyc_df         = pct_df(df["kyc_level"], "KYC Level") if "kyc_level" in df.columns else pd.DataFrame()
crisk_df       = pct_df(df["country_risk"], "Country Risk") if "country_risk" in df.columns else pd.DataFrame()
inc_df         = pct_df(df["income_bracket"], "Income Bracket") if "income_bracket" in df.columns else pd.DataFrame()
cust_risk_df   = pct_df(df["customer_risk_rating"], "Customer Risk Rating") if "customer_risk_rating" in df.columns else pd.DataFrame()
pep_df         = flag_summary(df["pep_flag"], "PEP Flag", {0:"Non-PEP",1:"PEP"}) if "pep_flag" in df.columns else pd.DataFrame()
industry_df    = pct_df(df["industry"], "Industry") if "industry" in df.columns else pd.DataFrame()
acc_type_df    = pct_df(df["account_type"], "Account Type") if "account_type" in df.columns else pd.DataFrame()

# ── 6. Device Profile ─────────────────────────────────────────────────────────
os_df          = pct_df(df["os_type"], "OS Type") if "os_type" in df.columns else pd.DataFrame()
rooted_df      = flag_summary(df["rooted_flag"], "Rooted", {0:"Not Rooted",1:"Rooted"}) if "rooted_flag" in df.columns else pd.DataFrame()
vpn_df         = flag_summary(df["vpn_flag"], "VPN", {0:"No VPN",1:"VPN Active"}) if "vpn_flag" in df.columns else pd.DataFrame()
emu_df         = flag_summary(df["emulator_flag"], "Emulator", {0:"Not Emulator",1:"Emulator"}) if "emulator_flag" in df.columns else pd.DataFrame()
dev_age_stats  = stats_df(df["device_age_days"], "Device Age (Days)") if "device_age_days" in df.columns else pd.DataFrame()

# ── 7. Beneficiary Profile ────────────────────────────────────────────────────
ben_type_df    = pct_df(df["beneficiary_type"], "Beneficiary Type") if "beneficiary_type" in df.columns else pd.DataFrame()
ben_risk_df    = pct_df(df["beneficiary_country_risk"], "Beneficiary Country Risk") if "beneficiary_country_risk" in df.columns else pd.DataFrame()
high_risk_b    = flag_summary(df["high_risk_beneficiary"], "High Risk Bene", {0:"Normal",1:"High Risk"}) if "high_risk_beneficiary" in df.columns else pd.DataFrame()

# ── 8. Engineered Features ────────────────────────────────────────────────────
#round_df       = flag_summary(df["is_round_amount"], "Round Amount", {0:"No",1:"Yes"}) if "is_round_amount" in df.columns else pd.DataFrame()
dormancy_df    = flag_summary(df["dormancy_flag"], "Dormancy", {0:"Active",1:"Dormant"}) if "dormancy_flag" in df.columns else pd.DataFrame()

# ── 9. Rule Engine ────────────────────────────────────────────────────────────
rule_cols = [c for c in df.columns if c.startswith("rule_") and c != "rule_trigger_count"]
rules_df  = rule_summary(rule_cols, df)

trigger_stats = stats_df(df["rule_trigger_count"], "Rule Trigger Count") if "rule_trigger_count" in df.columns else pd.DataFrame()
severity_df   = pct_df(df["max_rule_severity"], "Max Rule Severity") if "max_rule_severity" in df.columns else pd.DataFrame()
wscore_stats  = stats_df(df["weighted_rule_score"], "Weighted Rule Score") if "weighted_rule_score" in df.columns else pd.DataFrame()

# ── Fraud vs Legit rule comparison ───────────────────────────────────────────
if "label" in df.columns and rule_cols:
    fraud_rules = df[df["label"] == 1][rule_cols].mean().round(4)
    legit_rules = df[df["label"] == 0][rule_cols].mean().round(4)
    rule_compare = pd.DataFrame({
        "Rule": [c.replace("rule_","").replace("_"," ").title() for c in rule_cols],
        "Fraud Fire Rate": (fraud_rules.values * 100).round(2),
        "Legit Fire Rate": (legit_rules.values * 100).round(2),
        "Lift (Fraud/Legit)": (fraud_rules.values / (legit_rules.values + 1e-9)).round(2),
    }).sort_values("Lift (Fraud/Legit)", ascending=False).reset_index(drop=True)
else:
    rule_compare = pd.DataFrame()


# ── 10. Fraud Type Examples ────────────────────────────────────────────────────
# For each fraud_type, pick one representative transaction from the data and
# generate a human-readable reasoning string that explains WHY it is flagged.
#
# Signal definitions used in reasoning:
#   - amount          : raw INR amount
#   - cash_flag       : 1 = physical cash involved
#   - dormancy_flag   : 1 = account inactive >30 days before this transaction
#   - is_night        : 1 = transaction between 22:00–06:00
#   - is_weekend      : 1 = Saturday or Sunday
#   - kyc_level       : low / medium / high
#   - beneficiary_type: crypto / offshore = high-risk exit node
#   - country_risk    : high = FATF-listed or elevated jurisdiction
#   - txn_count_last_1hr : velocity in past 1 hour
#   - txn_count_last_24hr: velocity in past 24 hours
#   - pep_flag        : 1 = Politically Exposed Person
#   - rooted_flag     : 1 = rooted/jailbroken device
#   - vpn_flag        : 1 = VPN/proxy detected
#   - emulator_flag   : 1 = emulated environment

FRAUD_TYPE_DESCRIPTIONS = {
    "mule_ring": (
        "Mule Ring",
        "A network of accounts (mules) used to receive and rapidly forward illicit funds. "
        "Fraudsters recruit individuals to open accounts, receive deposits, and immediately "
        "transfer funds onward — fragmenting the money trail across multiple hops."
    ),
    "layering": (
        "Layering",
        "Funds are moved through a chain of transactions in rapid succession to obscure their "
        "origin. Each hop is slightly smaller than the last (decay pattern) and executed within "
        "minutes, making it hard to trace back to the original illicit source."
    ),
    "ATO": (
        "Account Takeover (ATO)",
        "A fraudster hijacks a legitimate customer's account — typically via credential stuffing "
        "or phishing — and immediately drains funds using unusual channels, devices, or times "
        "that differ from the account owner's normal behaviour."
    ),
    "smurfing": (
        "Smurfing (Structuring)",
        "Large amounts of cash are broken into multiple smaller transactions, each deliberately "
        "kept just below the ₹10,000 regulatory reporting threshold. The intent is to avoid "
        "triggering cash transaction reports (CTRs) mandated by anti-money-laundering rules."
    ),
    "identity_fraud": (
        "Identity Fraud",
        "A fraudulent account opened using stolen or synthetic identity documents. These accounts "
        "are typically new (< 60 days old) and immediately used for high-value transactions before "
        "the bank can detect the identity mismatch."
    ),
    "dormant_ATO": (
        "Dormant Account Takeover",
        "A dormant (inactive > 30 days) account is taken over by a fraudster using a new, "
        "unrecognised device. The reactivation burst — large amounts sent to high-risk beneficiaries "
        "immediately after a long silence — is a strong AML signal combining account takeover and "
        "dormancy reactivation."
    ),
    "dormant_smurfing": (
        "Dormant Account Smurfing",
        "A reactivated dormant account is used for cash structuring: multiple cash transactions "
        "just below the ₹10,000 threshold within a few days of the reactivation event. The "
        "combination of dormancy + structuring is one of the highest-priority FATF red flags."
    ),
    "dormant_to_offshore": (
        "Dormant Account — Offshore Wire",
        "Funds parked in a dormant account are suddenly wired offshore or to a crypto wallet "
        "immediately upon reactivation. This represents the integration step of a laundering "
        "scheme: the account was dormant during placement/layering and is now used for extraction."
    ),
}

def _val(row, col, default="N/A"):
    """Safe row value accessor."""
    v = row.get(col, default)
    return default if (v is None or (isinstance(v, float) and np.isnan(v))) else v

def build_fraud_reasoning(row, fraud_type):
    """
    Return a list of signal strings explaining why this transaction
    is flagged for the given fraud_type.
    """
    signals = []
    amt = _val(row, "amount", 0)

    if fraud_type == "mule_ring":
        signals.append(f"High-value transfer of ₹{amt:,.2f} routed through a mule account")
        if _val(row, "txn_count_last_24hr", 0) > 10:
            signals.append(f"Account processed {_val(row,'txn_count_last_24hr')} transactions in last 24hr — rapid forwarding pattern")
        if _val(row, "beneficiary_type") in ("crypto","offshore"):
            signals.append(f"Funds exit to a {_val(row,'beneficiary_type')} beneficiary — high-risk exit node")
        if _val(row, "is_night", 0) == 1:
            signals.append("Transaction executed at night (22:00–06:00) — outside normal banking hours")
        if _val(row, "customer_risk_rating") == "very_high":
            signals.append("Customer flagged as very-high risk in CDD rating")

    elif fraud_type == "layering":
        signals.append(f"Transaction of ₹{amt:,.2f} is part of a rapid multi-hop chain (layering)")
        if _val(row, "txn_count_last_1hr", 0) > 2:
            signals.append(f"{_val(row,'txn_count_last_1hr')} transactions fired in the last 1 hour — rapid hop sequence")
        if _val(row, "beneficiary_type") in ("crypto","offshore"):
            signals.append(f"Final hop exits to {_val(row,'beneficiary_type')} — obscures money trail")
        if _val(row, "channel") in ("mobile","web"):
            signals.append(f"Digital channel ({_val(row,'channel')}) used — consistent with automated layering scripts")
        if _val(row, "is_night", 0) == 1:
            signals.append("Executed at night — reduces chance of real-time human review")

    elif fraud_type == "ATO":
        signals.append(f"Unusually large withdrawal of ₹{amt:,.2f} on a potentially hijacked account")
        if _val(row, "device_age_days", 9999) < 90:
            signals.append(f"Device is only {_val(row,'device_age_days')} days old — unrecognised device signal")
        if _val(row, "rooted_flag", 0) == 1:
            signals.append("Device is rooted/jailbroken — malware risk elevated")
        if _val(row, "vpn_flag", 0) == 1:
            signals.append("VPN/proxy active — attacker masking true geolocation")
        if _val(row, "emulator_flag", 0) == 1:
            signals.append("Emulated environment detected — scripted fraud tooling likely")
        if _val(row, "is_night", 0) == 1:
            signals.append("Attack executed at night when account owner is unlikely to notice")

    elif fraud_type == "smurfing":
        signals.append(f"Cash transaction of ₹{amt:,.2f} — just below ₹10,000 CTR threshold (structuring)")
        if _val(row, "cash_flag", 0) == 1:
            signals.append("Cash flag = 1 confirms physical cash involved — harder to trace")
        if _val(row, "txn_count_last_7d", 0) > 3:
            signals.append(f"{_val(row,'txn_count_last_7d')} transactions in last 7 days — repeated sub-threshold pattern")
        if _val(row, "channel") in ("branch","atm"):
            signals.append(f"Channel = {_val(row,'channel')} — cash structuring typically happens at branches/ATMs")

    elif fraud_type == "identity_fraud":
        signals.append(f"High-value transaction of ₹{amt:,.2f} on a newly opened account")
        if _val(row, "account_open_days", 9999) < 60:
            signals.append(f"Account is only {_val(row,'account_open_days')} days old — synthetic identity accounts act fast")
        if _val(row, "kyc_level") == "low":
            signals.append("KYC level = low — minimal identity verification was performed at onboarding")
        if _val(row, "country_risk") == "high":
            signals.append("Customer from a high-risk jurisdiction — elevated identity fraud exposure")
        if _val(row, "beneficiary_type") in ("crypto","offshore"):
            signals.append(f"Funds sent to {_val(row,'beneficiary_type')} — fast extraction before account is frozen")

    elif fraud_type == "dormant_ATO":
        signals.append(f"Account was dormant (inactive >30 days) and reactivated with a ₹{amt:,.2f} transaction")
        if _val(row, "dormancy_flag", 0) == 1:
            signals.append("dormancy_flag = 1 — confirms this is the reactivation transaction after long silence")
        if _val(row, "device_age_days", 9999) < 90:
            signals.append(f"New device ({_val(row,'device_age_days')} days old) used — account owner would use their established device")
        if _val(row, "beneficiary_type") in ("crypto","offshore"):
            signals.append(f"Immediate transfer to {_val(row,'beneficiary_type')} — rapid fund extraction post-takeover")
        if _val(row, "is_night", 0) == 1:
            signals.append("Reactivation occurred at night — attacker avoids detection during business hours")

    elif fraud_type == "dormant_smurfing":
        signals.append(f"Dormant account reactivated and immediately used for cash structuring (₹{amt:,.2f} per transaction)")
        if _val(row, "dormancy_flag", 0) == 1:
            signals.append("dormancy_flag = 1 — this transaction follows a >30-day account silence")
        if _val(row, "cash_flag", 0) == 1:
            signals.append("Cash flag = 1 — physical cash structuring on a reactivated account")
        if 8500 <= float(amt or 0) <= 9999:
            signals.append(f"Amount ₹{amt:,.2f} is in ₹8,500–₹9,999 band — deliberate sub-threshold structuring")
        if _val(row, "txn_count_last_7d", 0) > 2:
            signals.append(f"{_val(row,'txn_count_last_7d')} similar transactions in last 7 days — repeated structuring pattern")

    elif fraud_type == "dormant_to_offshore":
        signals.append(f"Dormant account reactivated with a large offshore wire of ₹{amt:,.2f}")
        if _val(row, "dormancy_flag", 0) == 1:
            signals.append("dormancy_flag = 1 — account was silent for >30 days before this extraction")
        if _val(row, "beneficiary_type") in ("crypto","offshore"):
            signals.append(f"Beneficiary type = {_val(row,'beneficiary_type')} — integration step: extracting previously placed funds")
        if _val(row, "beneficiary_country_risk") == "high":
            signals.append("Destination country is high-risk jurisdiction — consistent with money laundering route")
        if _val(row, "amount_to_balance_ratio", 0) > 1:
            signals.append(f"Amount-to-balance ratio = {_val(row,'amount_to_balance_ratio',0):.2f} — transaction exceeds typical account balance")

    else:
        signals.append(f"Transaction of ₹{amt:,.2f} flagged as {fraud_type}")

    return signals

# ── Build per-fraud-type example rows ─────────────────────────────────────────
fraud_examples = {}  # fraud_type → (example_row_df, signals_list)

if "fraud_type" in df.columns and "label" in df.columns:
    for ftype in df[df["label"] == 1]["fraud_type"].dropna().unique():
        subset = df[(df["fraud_type"] == ftype) & (df["label"] == 1)]
        if subset.empty:
            continue

        # Pick the most "extreme" example: highest weighted_rule_score, then highest amount
        sort_cols = [c for c in ["weighted_rule_score", "amount"] if c in subset.columns]
        example_row = subset.sort_values(sort_cols, ascending=False).iloc[0]

        # Columns to show in the example table
        display_cols = [c for c in [
            "transaction_id", "sender_account_id", "customer_id",
            "timestamp", "amount", "channel", "transaction_type",
            "cash_flag", "dormancy_flag", "is_night", "is_weekend",
            "beneficiary_type", "beneficiary_country_risk",
            "kyc_level", "country_risk", "occupation",
            "device_age_days", "rooted_flag", "vpn_flag", "emulator_flag",
            "account_open_days", "customer_risk_rating",
            "txn_count_last_1hr", "txn_count_last_24hr", "txn_count_last_7d",
            "amount_to_balance_ratio", "weighted_rule_score",
            "fraud_type", "label",
        ] if c in subset.columns]

        example_df = example_row[display_cols].reset_index(drop=True).to_frame("Value")
        example_df.index.name = "Field"
        example_df = example_df.reset_index()
        example_df["Value"] = example_df["Value"].astype(str)

        signals = build_fraud_reasoning(example_row, ftype)
        fraud_examples[ftype] = (example_df, signals)



In [21]:

# Account-level dormancy: distinct accounts that have ≥1 dormancy_flag=1 transaction
if "dormancy_flag" in df.columns and "sender_account_id" in df.columns:
    total_accounts   = df["sender_account_id"].nunique()
    dormant_accounts = df[df["dormancy_flag"] == 1]["sender_account_id"].nunique()
    active_accounts  = total_accounts - dormant_accounts
    dormant_accounts_df = pd.DataFrame({
        "Account Status": ["Dormant (≥1 reactivation txn)", "Active (no dormancy flag)", "Total Unique Accounts"],
        "Account Count":  [dormant_accounts, active_accounts, total_accounts],
        "Percentage (%)": [
            round(dormant_accounts / total_accounts * 100, 2),
            round(active_accounts  / total_accounts * 100, 2),
            100.0,
        ],
    })
else:
    dormant_accounts_df = pd.DataFrame()
ratio_stats    = stats_df(df["amount_to_balance_ratio"], "Amount-to-Balance Ratio") if "amount_to_balance_ratio" in df.columns else pd.DataFrame()
log_amt_stats  = stats_df(df["log_amount"], "Log Amount") if "log_amount" in df.columns else pd.DataFrame()



In [23]:
# ══════════════════════════════════════════════════════════════════════════════
# BUILD EXCEL WORKBOOK
# ══════════════════════════════════════════════════════════════════════════════

wb = Workbook()

# ─── SHEET 1: Dashboard ───────────────────────────────────────────────────────
ws0 = wb.active
ws0.title = "📊 Dashboard"
ws0.sheet_view.showGridLines = False

ws0.merge_cells("A1:F1")
hdr(ws0, 1, 1, "SmartSentry AML — Transaction Data Summary Report", size=15, fill=HDR_FILL)
ws0.row_dimensions[1].height = 40
ws0.merge_cells("A2:F2")
hdr(ws0, 2, 1, f"Total Transactions: {TOTAL:,}  |  Fraud Rate: {fraud_pct}%  |  Unique Customers: {df['customer_id'].nunique():,}", size=10, fill=SUB_FILL)

r = 4
r = write_block(ws0, r, 1, "📌 Key Performance Indicators", overview_kpis)
if not fraud_type_df.empty:
    r = write_block(ws0, r, 1, "🎭 Fraud Type Breakdown", fraud_type_df, flag_high=5.0)

# ─── SHEET 2: Transactions ────────────────────────────────────────────────────
ws1 = wb.create_sheet("💳 Transactions")
ws1.sheet_view.showGridLines = False
ws1.merge_cells("A1:J1")
hdr(ws1, 1, 1, "TRANSACTION CHARACTERISTICS", size=13, fill=HDR_FILL)
ws1.row_dimensions[1].height = 35

r = 3
r = write_block(ws1, r, 1, "Amount Statistics (INR)", amount_stats)
r = write_block(ws1, r, 1, "Channel Distribution", channel_df)
r = write_block(ws1, r, 1, "Transaction Type", txn_type_df)

r2 = 3
r2 = write_block(ws1, r2, 5, "Debit / Credit Split", debit_df)
r2 = write_block(ws1, r2, 5, "Cash Flag", cash_df)
#r2 = write_block(ws1, r2, 5, "Round Amount", round_df)
r2 = write_block(ws1, r2, 5, "Dormancy Flag (Transactions)", dormancy_df)
r2 = write_block(ws1, r2, 5, "Dormancy by Account", dormant_accounts_df)

r3 = 3
r3 = write_block(ws1, r3, 9, "Amount-to-Balance Ratio Stats", ratio_stats)
r3 = write_block(ws1, r3, 9, "Log Amount Stats", log_amt_stats)

# ─── SHEET 3: Temporal ────────────────────────────────────────────────────────
ws2 = wb.create_sheet("⏰ Temporal")
ws2.sheet_view.showGridLines = False
ws2.merge_cells("A1:J1")
hdr(ws2, 1, 1, "TEMPORAL DISTRIBUTION", size=13, fill=HDR_FILL)
ws2.row_dimensions[1].height = 35

r = 3
r = write_block(ws2, r, 1, "Hour of Day Distribution", hour_df)
r = write_block(ws2, r, 1, "Day of Week Distribution", dow_df)
r = write_block(ws2, r, 1, "Month Distribution", month_df)

r2 = 3
r2 = write_block(ws2, r2, 5, "Is Night Transaction", night_df)
r2 = write_block(ws2, r2, 5, "Is Weekend Transaction", weekend_df)
r2 = write_block(ws2, r2, 5, "Is Business Hours", biz_df)

# ─── SHEET 4: Customer & Account ─────────────────────────────────────────────
ws3 = wb.create_sheet("👤 Customer & Account")
ws3.sheet_view.showGridLines = False
ws3.merge_cells("A1:J1")
hdr(ws3, 1, 1, "CUSTOMER & ACCOUNT PROFILE", size=13, fill=HDR_FILL)
ws3.row_dimensions[1].height = 35

r = 3
r = write_block(ws3, r, 1, "Occupation", occ_df)
r = write_block(ws3, r, 1, "Industry", industry_df)
r = write_block(ws3, r, 1, "Account Type", acc_type_df)

r2 = 3
r2 = write_block(ws3, r2, 5, "KYC Level", kyc_df)
r2 = write_block(ws3, r2, 5, "Country Risk", crisk_df)
r2 = write_block(ws3, r2, 5, "Customer Risk Rating", cust_risk_df)
r2 = write_block(ws3, r2, 5, "Income Bracket", inc_df)
r2 = write_block(ws3, r2, 5, "PEP Flag", pep_df)

# ─── SHEET 5: Devices ────────────────────────────────────────────────────────
ws4 = wb.create_sheet("📱 Devices")
ws4.sheet_view.showGridLines = False
ws4.merge_cells("A1:J1")
hdr(ws4, 1, 1, "DEVICE PROFILE", size=13, fill=HDR_FILL)
ws4.row_dimensions[1].height = 35

r = 3
r = write_block(ws4, r, 1, "OS Type Distribution", os_df)
r = write_block(ws4, r, 1, "Rooted Device", rooted_df, flag_high=3.0)
r = write_block(ws4, r, 1, "VPN Active", vpn_df, flag_high=5.0)
r = write_block(ws4, r, 1, "Emulator Detected", emu_df, flag_high=2.0)

r2 = 3
r2 = write_block(ws4, r2, 5, "Device Age (Days) Statistics", dev_age_stats)

# ─── SHEET 6: Beneficiaries ───────────────────────────────────────────────────
ws5 = wb.create_sheet("💸 Beneficiaries")
ws5.sheet_view.showGridLines = False
ws5.merge_cells("A1:J1")
hdr(ws5, 1, 1, "BENEFICIARY PROFILE", size=13, fill=HDR_FILL)
ws5.row_dimensions[1].height = 35

r = 3
r = write_block(ws5, r, 1, "Beneficiary Type", ben_type_df)
r = write_block(ws5, r, 1, "Beneficiary Country Risk", ben_risk_df, flag_high=5.0)
r = write_block(ws5, r, 1, "High Risk Beneficiary Flag", high_risk_b, flag_high=5.0)

# ─── SHEET 7: Velocity Features ───────────────────────────────────────────────
ws6 = wb.create_sheet("⚡ Velocity Features")
ws6.sheet_view.showGridLines = False
ws6.merge_cells("A1:K1")
hdr(ws6, 1, 1, "VELOCITY & ROLLING WINDOW FEATURE STATISTICS", size=13, fill=HDR_FILL)
ws6.row_dimensions[1].height = 35

r = 3
# if not velocity_stats.empty:
#     r = write_block(ws6, r, 1, "Account & Customer Velocity Feature Statistics", velocity_stats)

# ─── SHEET 8: Rule Engine ─────────────────────────────────────────────────────
ws7 = wb.create_sheet("🔴 Rule Engine")
ws7.sheet_view.showGridLines = False
ws7.merge_cells("A1:J1")
hdr(ws7, 1, 1, "AML RULE ENGINE — TRIGGER ANALYSIS", size=13, fill=HDR_FILL)
ws7.row_dimensions[1].height = 35

r = 3
if not rules_df.empty:
    r = write_block(ws7, r, 1, "All Rules — Trigger Frequency (sorted by volume)", rules_df)

r2 = 3
if not trigger_stats.empty:
    r2 = write_block(ws7, r2, 6, "Rule Trigger Count Statistics", trigger_stats)
if not severity_df.empty:
    r2 = write_block(ws7, r2, 6, "Max Rule Severity Distribution", severity_df)
if not wscore_stats.empty:
    r2 = write_block(ws7, r2, 6, "Weighted Rule Score Statistics", wscore_stats)

# ─── SHEET 9: Rule Lift (Fraud vs Legit) ──────────────────────────────────────
if not rule_compare.empty:
    ws8 = wb.create_sheet("📈 Rule Lift Analysis")
    ws8.sheet_view.showGridLines = False
    ws8.merge_cells("A1:E1")
    hdr(ws8, 1, 1, "RULE LIFT ANALYSIS — Fraud Fire Rate vs Legitimate Fire Rate", size=13, fill=HDR_FILL)
    ws8.merge_cells("A2:E2")
    hdr(ws8, 2, 1, "Lift > 1 means rule fires more on fraud. Higher lift = stronger discriminating power.", size=9, fill=SUB_FILL)
    ws8.row_dimensions[1].height = 35
    write_block(ws8, 4, 1, "Rule Discrimination Power (sorted by Lift)", rule_compare)


# ─── SHEET 10: Fraud Type Examples ────────────────────────────────────────────
if fraud_examples:
    ws9 = wb.create_sheet("🔍 Fraud Examples")
    ws9.sheet_view.showGridLines = False

    # Wide header
    ws9.merge_cells("A1:B1")
    hdr(ws9, 1, 1,
        "FRAUD TYPOLOGY EXAMPLES — One Representative Transaction Per Type",
        size=13, fill=HDR_FILL)
    ws9.row_dimensions[1].height = 35

    ws9.merge_cells("A2:B2")
    hdr(ws9, 2, 1,
        "Each block shows the highest-risk example transaction for that fraud type, "
        "the signals that triggered it, and a plain-English reasoning.",
        size=9, fill=SUB_FILL)
    ws9.row_dimensions[2].height = 28

    # Write left column (col 1) and right column (col 3) interleaved
    # to use the sheet width efficiently
    fraud_types_ordered = [
        "mule_ring", "layering", "ATO", "smurfing", "identity_fraud",
        "dormant_ATO", "dormant_smurfing", "dormant_to_offshore",
    ]
    # Fall back to whatever order exists if a type is missing
    ordered_keys = [k for k in fraud_types_ordered if k in fraud_examples]
    ordered_keys += [k for k in fraud_examples if k not in ordered_keys]

    r_left  = 4   # left column (col 1)
    r_right = 4   # right column (col 4)
    for i, ftype in enumerate(ordered_keys):
        example_df, signals = fraud_examples[ftype]
        if i % 2 == 0:
            r_left = write_fraud_example(
                ws9, r_left, 1, ftype, example_df, signals, FRAUD_TYPE_DESCRIPTIONS
            )
        else:
            r_right = write_fraud_example(
                ws9, r_right, 4, ftype, example_df, signals, FRAUD_TYPE_DESCRIPTIONS
            )



# ─── SHEET 11: Fraud Transaction Examples — Wide Format ──────────────────────
# Structure:
#   PART A — 2 examples per fraud type, each with its own reasoning row
#   PART B — Full deep-dive: ALL transactions for "layering" (or first type)
#
# device_id added to WIDE_COLS for device-level visibility
WIDE_COLS = [c for c in [
    "transaction_id", "sender_account_id", "device_id", "customer_id",
    "timestamp", "amount", "channel", "transaction_type", "debit_credit",
    "cash_flag", "dormancy_flag", "is_night", "is_weekend",
    "beneficiary_id", "beneficiary_type", "beneficiary_country_risk",
    "high_risk_beneficiary",
    "kyc_level", "country_risk", "occupation", "income_bracket",
    "customer_risk_rating", "pep_flag", "account_type",
    "device_age_days", "rooted_flag", "vpn_flag", "emulator_flag",
    "account_open_days",
    "txn_count_last_1hr", "txn_count_last_24hr", "txn_count_last_7d",
    "total_amount_last_24hr", "total_amount_last_7d",
    "amount_to_balance_ratio", "log_amount", "is_round_amount",
    "weighted_rule_score", "rule_trigger_count", "max_rule_severity",
    "fraud_type", "label",
] if c in df.columns]

TYPE_COLOURS     = ["C00000","7030A0","0070C0","FF8C00","00B050","C9511F","833C00","4472C4"]
TYPE_HDR2_COLOURS= ["FFD7D7","E8D5F5","D0E8F7","FFE8C8","D5F0DC","F5D7C0","EAD6C0","D0DCF5"]
ALT_COLOURS      = ["FFF0F0","F7F0FF","EEF6FF","FFF5E8","EEFAF2","FDF3EC","F7EDE3","EEF2FF"]

def _write_row_data(ws, cur_row, row_data, wide_cols, fill):
    for ci, col in enumerate(wide_cols, start=1):
        val = row_data[col] if col in row_data.index else ""
        if isinstance(val, float):
            if col in ("amount",) or col.startswith("total_amount"):
                val = round(val, 2)
            elif col in ("amount_to_balance_ratio", "log_amount"):
                val = round(val, 4)
        c = ws.cell(row=cur_row, column=ci, value=val)
        c.font      = Font(name="Arial", size=9, color="1F2D3D")
        c.fill      = fill
        c.alignment = Alignment(horizontal="left" if ci==1 else "center", vertical="center")
        c.border    = THIN
    ws.row_dimensions[cur_row].height = 14
    return cur_row + 1

def _write_col_headers(ws, cur_row, wide_cols, hdr2_fill):
    for ci, col_name in enumerate(wide_cols, start=1):
        hdr(ws, cur_row, ci, col_name.replace("_"," ").title(), size=9, fill=hdr2_fill, color="1F2D3D")
    ws.row_dimensions[cur_row].height = 28
    return cur_row + 1

def _write_banner(ws, cur_row, text, n_cols, hex_color, height=26):
    ws.merge_cells(start_row=cur_row, start_column=1, end_row=cur_row, end_column=n_cols)
    c = ws.cell(row=cur_row, column=1, value=text)
    c.font      = Font(name="Arial", bold=True, size=12, color="FFFFFF")
    c.fill      = PatternFill("solid", start_color=hex_color)
    c.alignment = Alignment(horizontal="left", vertical="center")
    c.border    = THIN
    ws.row_dimensions[cur_row].height = height
    return cur_row + 1

def _write_info(ws, cur_row, text, n_cols, bg_hex, italic=False, size=9, height=34):
    ws.merge_cells(start_row=cur_row, start_column=1, end_row=cur_row, end_column=n_cols)
    c = ws.cell(row=cur_row, column=1, value=text)
    c.font      = Font(name="Arial", italic=italic, size=size, color="1F2D3D")
    c.fill      = PatternFill("solid", start_color=bg_hex)
    c.alignment = Alignment(horizontal="left", vertical="center", wrap_text=True)
    c.border    = THIN
    ws.row_dimensions[cur_row].height = height
    return cur_row + 1

def _autosize(ws, wide_cols, cur_row):
    for ci, col in enumerate(wide_cols, start=1):
        cl     = get_column_letter(ci)
        header = col.replace("_"," ").title()
        sample = [header] + [str(ws.cell(r,ci).value or "") for r in range(1, min(cur_row,120))]
        ws.column_dimensions[cl].width = min(max(max(len(v) for v in sample)+3, 12), 38)



ValueError: 8 must be greater than 9

In [15]:
if "fraud_type" in df.columns and "label" in df.columns:
    ws10 = wb.create_sheet("📋 Fraud Txn Wide Table")
    ws10.sheet_view.showGridLines = False
    ws10.freeze_panes = "B1"

    fraud_types_ordered = [
        "mule_ring","layering","ATO","smurfing","identity_fraud",
        "dormant_ATO","dormant_smurfing","dormant_to_offshore",
    ]
    ordered_wide  = [k for k in fraud_types_ordered if k in df["fraud_type"].values]
    ordered_wide += [k for k in df[df["label"]==1]["fraud_type"].dropna().unique()
                     if k not in ordered_wide]
    n_cols   = len(WIDE_COLS)
    cur_row  = 1

    # ══════════════════════════════════════════════════════════════════════
    # PART A — 2 examples per fraud type, each followed by its reasoning row
    # ══════════════════════════════════════════════════════════════════════
    cur_row = _write_banner(ws10, cur_row,
        "PART A — 2 Representative Examples Per Fraud Type  "
        "(Example 1 = highest risk score · Example 2 = second highest)",
        n_cols, "1F3864", height=30)
    cur_row = _write_info(ws10, cur_row,
        "Each example transaction is followed immediately by an ⚠️ reasoning row "
        "that explains which specific field values triggered the AML alert for that fraud type. "
        "device_id is included so you can track device-level reuse patterns.",
        n_cols, "BDD7EE", height=30)
    cur_row += 1

    for type_idx, ftype in enumerate(ordered_wide):
        ci_       = type_idx % len(TYPE_COLOURS)
        ban_hex   = TYPE_COLOURS[ci_]
        hdr2_hex  = TYPE_HDR2_COLOURS[ci_]
        alt_hex   = ALT_COLOURS[ci_]
        ALT_T     = PatternFill("solid", start_color=alt_hex)
        HDR2_T    = PatternFill("solid", start_color=hdr2_hex)

        display_name, description = FRAUD_TYPE_DESCRIPTIONS.get(
            ftype, (ftype.replace("_"," ").title(), ""))
        subset      = df[(df["fraud_type"]==ftype) & (df["label"]==1)].copy()
        total_count = len(subset)
        sort_by     = [c for c in ["weighted_rule_score","amount"] if c in subset.columns]
        top2        = subset.sort_values(sort_by, ascending=False).head(2)

        # Banner
        cur_row = _write_banner(ws10, cur_row,
            f"🔴  {display_name.upper()}   (showing 2 of {total_count:,} total transactions)",
            n_cols, ban_hex)
        # Description
        cur_row = _write_info(ws10, cur_row, description, n_cols, "FFF2CC",
                              italic=True, height=38)
        # Column headers
        cur_row = _write_col_headers(ws10, cur_row, WIDE_COLS, HDR2_T)

        # 2 examples each with reasoning
        for ex_num, (_, ex_row) in enumerate(top2.iterrows(), start=1):
            fill    = WHITE if ex_num == 1 else ALT_T
            cur_row = _write_row_data(ws10, cur_row, ex_row, WIDE_COLS, fill)

            signals    = build_fraud_reasoning(ex_row, ftype)
            signal_txt = "  ✦  ".join(signals)
            cur_row = _write_info(ws10, cur_row,
                f"  ⚠️  Example {ex_num} reasoning:  {signal_txt}",
                n_cols, "FFE0B2", height=32)

        cur_row += 2   # gap between fraud types

    # ══════════════════════════════════════════════════════════════════════
    # PART B — Full deep-dive: ALL transactions for one fraud type
    # ══════════════════════════════════════════════════════════════════════
    cur_row += 1
    deep_type   = "layering" if "layering" in ordered_wide else ordered_wide[0]
    dd_name, dd_desc = FRAUD_TYPE_DESCRIPTIONS.get(
        deep_type, (deep_type.replace("_"," ").title(), ""))
    dd_ci    = ordered_wide.index(deep_type) % len(TYPE_COLOURS)
    DD_BAN   = TYPE_COLOURS[dd_ci]
    DD_HDR2  = PatternFill("solid", start_color=TYPE_HDR2_COLOURS[dd_ci])
    DD_ALT   = PatternFill("solid", start_color=ALT_COLOURS[dd_ci])

    dd_df    = df[(df["fraud_type"]==deep_type) & (df["label"]==1)][WIDE_COLS].copy()
    sort_by  = [c for c in ["weighted_rule_score","amount"] if c in dd_df.columns]
    dd_df    = dd_df.sort_values(sort_by, ascending=False).reset_index(drop=True)

    cur_row = _write_banner(ws10, cur_row,
        f"PART B — Complete Deep-Dive: {dd_name.upper()}  "
        f"({len(dd_df):,} transactions · sorted by risk score descending)",
        n_cols, "1F3864", height=30)
    cur_row = _write_info(ws10, cur_row,
        f"What is {dd_name}?  {dd_desc}  "
        f"All {len(dd_df):,} transactions are listed below in descending risk order so you can "
        f"examine the full range of amounts, timing patterns, device IDs, and beneficiary types "
        f"associated with this fraud type.",
        n_cols, "BDD7EE", height=48)
    cur_row = _write_col_headers(ws10, cur_row, WIDE_COLS, DD_HDR2)

    for ri, (_, row_data) in enumerate(dd_df.iterrows()):
        cur_row = _write_row_data(ws10, cur_row, row_data, WIDE_COLS,
                                  DD_ALT if ri % 2 == 1 else WHITE)

    _autosize(ws10, WIDE_COLS, cur_row)

    print(f"   📋 Fraud Txn Wide Table: "
          f"Part A ({len(ordered_wide)} types × 2 examples) + "
          f"Part B (all {len(dd_df):,} '{deep_type}' rows)")
# ── Save ───────────────────────────────────────────────────────────────────────
wb.save(OUTPUT_PATH)
print(f"\n✅ Summary saved to: {OUTPUT_PATH}")
print(f"   Sheets: {[ws.title for ws in wb.worksheets]}")

NameError: name 'WIDE_COLS' is not defined